Ref: https://github.com/NCAR/cesm-lens-aws/issues/34

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import xarray as xr
import intake
from tqdm.auto import tqdm
import dask
from ncar_jobqueue import NCARCluster
from distributed import Client
from utils import _restore_non_dim_coords, preprocess, show_ds_info, print_ds_info, save_data, zarr_store
xr.set_options(display_style='html')
dask.config.set({'distributed.dashboard.link': '/proxy/{port}/status'})
dask.config.get('distributed.dashboard')

{'link': '/proxy/{port}/status', 'export-tool': False}

In [3]:
cluster = NCARCluster(cores=2)
#cluster.adapt(minimum_jobs=2, maximum_jobs=80, wait_count=120)
cluster.scale(80)
client = Client(cluster)
cluster

In [22]:
col = intake.open_esm_datastore("../catalogs/glade-campaign-cesm1-le.json")
col

glade-cesm1-le-ESM Collection with 146462 entries:
	> 6 experiment(s)

	> 93 case(s)

	> 6 component(s)

	> 13 stream(s)

	> 820 variable(s)

	> 89 date_range(s)

	> 41 member_id(s)

	> 146462 path(s)

	> 1 ctrl_branch_year(s)

	> 1 year_offset(s)

In [23]:
chunksCTRL = {'chunks': {'time': 360, 'z_t': 1,  'z_w_top': 1,  'z_w_bot': 1}}
chunksOther = {'chunks': {'member_id': 40, 'time': 12, 'z_t': 1,  'z_w_top': 1,  'z_w_bot': 1}}

In [24]:
variables = ["TEMP", "UVEL", "VVEL", "WVEL", "VNS", "VNT", "SHF", "SFWF"]
col_subset = col.search(variable=variables, experiment='20C')
col_subset

glade-cesm1-le-ESM Collection with 240 entries:
	> 1 experiment(s)

	> 40 case(s)

	> 1 component(s)

	> 1 stream(s)

	> 6 variable(s)

	> 2 date_range(s)

	> 40 member_id(s)

	> 240 path(s)

	> 1 ctrl_branch_year(s)

	> 0 year_offset(s)

In [25]:
chunks = {'time': 240, 'z_t': 1,  'z_w_top': 1,  'z_w_bot': 1}
chunksOut = {'member_id': 2, 'time': 240, 'z_t': 1,  'z_w_top': 1,  'z_w_bot': 1}
datasets = col_subset.to_dataset_dict(cdf_kwargs={'chunks': chunks}, preprocess=preprocess)

for key, ds in datasets.items():
    ds = _restore_non_dim_coords(ds)
    ds = ds.chunk(chunksOut)
    datasets[key] = ds
    variable = key.split('.')[-1]
    print_ds_info(ds, variable)
    print('\n')


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.variable'
                
--> There is/are 6 group(s)
Variable name: SFWF
Dataset dimensions: ('member_id', 'time', 'nlat', 'nlon')
Chunk shape: (2, 240, 384, 320)
Dataset shape: (40, 1872, 384, 320)
Chunk size: 235.93 MB
Dataset size: 36.83 GB


Variable name: SHF
Dataset dimensions: ('member_id', 'time', 'nlat', 'nlon')
Chunk shape: (2, 240, 384, 320)
Dataset shape: (40, 1872, 384, 320)
Chunk size: 235.93 MB
Dataset size: 36.83 GB


Variable name: TEMP
Dataset dimensions: ('member_id', 'time', 'z_t', 'nlat', 'nlon')
Chunk shape: (2, 240, 1, 384, 320)
Dataset shape: (40, 1872, 60, 384, 320)
Chunk size: 235.93 MB
Dataset size: 2.21 TB


Variable name: UVEL
Dataset dimensions: ('member_id', 'time', 'z_t', 'nlat', 'nlon')
Chunk shape: (2, 240, 1, 384, 320)
Dataset shape: (40, 1872, 60, 384, 320)
Chunk size: 235.93 MB
Dataset size: 2.21 TB


Variable name: VVEL
Dataset dimension

In [26]:
datasets.keys()

dict_keys(['ocn.20C.pop.h.SFWF', 'ocn.20C.pop.h.SHF', 'ocn.20C.pop.h.TEMP', 'ocn.20C.pop.h.UVEL', 'ocn.20C.pop.h.VVEL', 'ocn.20C.pop.h.WVEL'])

In [27]:
datasets['ocn.20C.pop.h.TEMP']

<xarray.Dataset>
Dimensions:               (d2: 2, lat_aux_grid: 395, member_id: 40, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, time: 1872, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * moc_z                 (moc_z) float32 0.0 1000.0 ... 525000.94 549999.06
  * z_w                   (z_w) float32 0.0 1000.0 2000.0 ... 500004.7 525000.94
  * z_w_bot               (z_w_bot) float32 1000.0 2000.0 ... 549999.06
  * z_w_top               (z_w_top) float32 0.0 1000.0 ... 500004.7 525000.94
  * z_t                   (z_t) float32 500.0 1500.0 ... 512502.8 537500.0
  * z_t_150m              (z_t_150m) float32 500.0 1500.0 ... 13500.0 14500.0
  * lat_aux_grid          (lat_aux_grid) float32 -79.48815 -78.952896 ... 90.0
  * time                  (time) object 1850-02-01 00:00:00 ... 2006-01-01 00:00:00
  * member_id             (member_id) int64 1 2 3 4 5 6 ... 101 102 103 104 105
Dimensions without coordinates: d2, moc_comp, nlat, nlon, transport_comp, transport_reg
Data variables:
    stefan_boltzmann      float64 ...
    rho_fw                float64 ...
    dz                    (z_t) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    hflux_factor          float64 ...
    latent_heat_fusion    float64 ...
    T0_Kelvin             float64 ...
    sea_ice_salinity      float64 ...
    DXU                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TAREA                 (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    heat_to_PW            float64 ...
    TLONG                 (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    time_bound            (time, d2) object dask.array<chunksize=(240, 2), meta=np.ndarray>
    KMU                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    omega                 float64 ...
    DXT                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    days_in_norm_year     timedelta64[ns] ...
    latent_heat_vapor     float64 ...
    HU                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    grav                  float64 ...
    ANGLET                (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    DYU                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    sound                 float64 ...
    transport_components  (transport_comp) |S256 dask.array<chunksize=(5,), meta=np.ndarray>
    UAREA                 (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    DYT                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    HUW                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    dzw                   (z_w) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    nsurface_u            float64 ...
    ppt_to_salt           float64 ...
    ULAT                  (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    radius                float64 ...
    HTE                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    rho_air               float64 ...
    sflux_factor          float64 ...
    salt_to_ppt           float64 ...
    nsurface_t            float64 ...
    ANGLE                 (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    salt_to_Svppt         float64 ...
    HUS                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    cp_air                float64 ...
    salinity_factor       float64 ...
    transport_regions     (transport_reg) |S256 dask.array<chunksize=(2,), meta=np.ndarray>
    REGION_MASK           (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    momentum_factor       float64 ...
    vonkar                float64 ...
    TLAT               

In [19]:
dirout = "/glade/scratch/abanihi/lens-aws"
for key, ds in tqdm(datasets.items()):
    key = key.split('.')
    exp, cmp, var, frequency = key[1], key[0], key[-1], 'monthly'
    store = zarr_store(exp, cmp, frequency, var, write=True, dirout=dirout)
    save_data(ds, store)

/glade/scratch/abanihi/lens-aws/ocn/monthly/cesmLE-20C-SFWF.zarr
/glade/scratch/abanihi/lens-aws/ocn/monthly/cesmLE-20C-SHF.zarr
/glade/scratch/abanihi/lens-aws/ocn/monthly/cesmLE-20C-TEMP.zarr
/glade/scratch/abanihi/lens-aws/ocn/monthly/cesmLE-20C-UVEL.zarr
/glade/scratch/abanihi/lens-aws/ocn/monthly/cesmLE-20C-VVEL.zarr
/glade/scratch/abanihi/lens-aws/ocn/monthly/cesmLE-20C-WVEL.zarr



In [ ]:
cluster.close()